### Load Poetry Dataset

In [4]:
# pip install openpyxl

In [1]:
import pandas as pd 
import random
from matplotlib import pyplot as plt

In [2]:
# pip install matplotlib

In [3]:
# load excel data
df = pd.read_excel('../Excel_files/Full_Poem_Dataset_12-17.xlsx')
df = df.drop(columns=['Unnamed: 0'])
records = df.to_dict('records')
random.choice(records)

{'Text': 'согревает этот смертный грех\nзнать о всех молчать о всех стихами\nтяжело разделывать орех\nв бычьей буче и ее рогани\n \nзатянули люди пояса\nи пробили дырочку металлом\nвыходили так на полчаса\nтолько навсегда уже не стало\n \nстало быть у выживших детей\nкак в матрешке будет по ребенку\nумирают люди без затей\nпробивают жизни перепонку\n \nулетают чтобы не смотреть\nна свои расхристанные плоти\nпусть другие переходят в смерть\nобрываясь на высокой ноте\n \nна высокой\n \nнизкие уйдут\nпропадут истлеют без остатка\nно оставят по себе редут\nда рубец\nнавроде отпечатка',
 'Author': 'Алексей Мишуков',
 'Before or after': 'After',
 'Source': 'No War Poetry',
 'Date posted': 'None',
 'UniqueIndex': 2013}

In [6]:
# pip install transformers

In [4]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
# % matplotlib inline

# Load pre-trained model tokenizer (vocabulary)

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

In [5]:
text = random.choice(random.choice(records)['Text'].split('\n'))
print(text)

marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

что могут слова сегодня
['[CLS]', 'что', 'могут', 'слова', 'сегодня', '[SEP]']


In [6]:
random.sample(list(tokenizer.vocab.keys()),10)

['Александровская',
 '섭',
 'Уставом',
 'Визант',
 '铁',
 'Кэмер',
 'резервную',
 'Гош',
 'жизни',
 'заплатив']

In [129]:
# Define a new example sentence with multiple meanings of the word "bank"
text = random.choice(random.choice(records)['Text'].split('\n'))
print(text)

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

воды солёной вслушайся,
[CLS]           101
воды         12,240
солё         46,590
##ной         1,700
всл          72,498
##уша        29,991
##й             860
##ся          1,523
,               128
[SEP]           102


In [7]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1]


In [8]:
tokenizer('следовательно')

{'input_ids': [101, 29761, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [17]:
# pip install spacy

In [9]:
import spacy 
# !python -m spacy download ru_core_news_lg
nlp = spacy.load("ru_core_news_lg")

In [10]:
from tqdm import tqdm

In [16]:
# Convert inputs to PyTorch tensors
lemmaCts = dict()
data = []
for rec in tqdm(records):
    lines = rec['Text'].split('\n')
    for line in lines:
        lemmas = [l.lemma_ for l in nlp(line)]
        for l in lemmas:
            lemmaCts.setdefault(l, 0)
            lemmaCts[l] += 1
        
        if len(line.split(' ')) < 5:
            continue
            
        lemmatized_line = ' '.join(lemmas)
        doc = tokenizer.encode_plus(lemmatized_line)

        data.append({
            'doc' : doc,
            'tokens' : tokenizer.convert_ids_to_tokens(doc['input_ids']),
            'text' : line,
            'lemmatized_text' : lemmatized_line,
            'rec' : rec
        })

100%|██████████████████████████████████████| 3222/3222 [08:01<00:00,  6.70it/s]


In [12]:
random.choice(data)

{'doc': {'input_ids': [101, 104713, 21547, 1469, 10226, 851, 24034, 1516, 40165, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 'tokens': ['[CLS]',
  'расколот',
  '##ься',
  'на',
  'свет',
  'и',
  'шум',
  'по',
  'рука',
  '[SEP]'],
 'text': 'раскололись на свет и шумы по рукам',
 'lemmatized_text': 'расколоться на свет и шум по рука',
 'rec': {'Text': 'вот что\nбывшие вещи мои закатились пропали по швам\nраскололись на свет и шумы по рукам\nразошлись по шкафам истлевают в дали\nзалегают в песке непорочные платья мои\nу чужих заседают непрочные книги мои\nразвеваются в реках\nподобные травам чулки\nпереплавлены в рельсы коньки транспортиры вязального детства крючки\nи какая по счёту петля хоть убей\nне припомню молекул самих по себе\nиз себя состоящих поток\nиз каракуль кометы мелком\nпо стеклу неподвижного зрения в мире другом',
  'Author': 'Хельга Ольшванг',
  'Before or after': 'Before',
  'Source': 'essentialpoetry',


In [13]:
import pickle

with open('../../records4RuBerta1-30.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [117]:
# !pip3 install pickle5
import pickle5 as pickle

with open('../../records4RuBerta1-30.pickle', 'rb') as handle:
    data = pickle.load(handle)
    
random.choice(data)

{'doc': {'input_ids': [101, 13164, 55456, 130, 96377, 93258, 13661, 128, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]},
 'tokens': ['[CLS]',
  'главный',
  'выигрыш',
  '-',
  'уметь',
  'обмануть',
  'правило',
  ',',
  '[SEP]'],
 'text': 'Главный выигрыш - уметь обмануть правила,',
 'lemmatized_text': 'главный выигрыш - уметь обмануть правило ,',
 'rec': {'Text': 'ПАМЯТИ НИКОЛАЯ АБАЕВА**\n**\nСиликоновые импланты, яхты миллиардеров,\nЗагар актеров на фоне прибоя,\nСтатистика короновируса, статистика футбольных матчей,\nКокорин, забивающий первый гол за Спартак,\nИ рядом разлагающийся труп тенгрианца Николая Абаева,\nАвтора монографии о психологических аспектах дзен буддизма,\nУдивительной по чистоте мысли и ясности слога.\nТираж этой книги навсегда остался в рюкзаках 80 х,\nНа перегонах между Ялтой и Марракешем,\nМежду Кяхтой и Джорданвиллем.\nВероятно, Абаев покончил с собой в день своего рождения,\nКогда ему исполнился 71 год,\n

In [14]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

Next, let's evaluate BERT on our example text, and fetch the hidden states of the network!

*Side note: `torch.no_grad` tells PyTorch not to construct the compute graph during this forward pass (since we won't be running backprop here)--this just reduces memory consumption and speeds things up a little.*


### Sample

In [19]:
ns = [50,100,200]
thresholdedLists = dict()
for n in ns:
    thresholdedLists[n] = []
    
for l in lemmaCts:
    for n in ns:
        if lemmaCts[l] > n:
            thresholdedLists[n].append(l)

In [20]:
len(thresholdedLists[200])

247

In [70]:
keyword = random.choice(thresholdedLists[200])
keyword = 'мир'
print(keyword)
sample = [d for d in data if keyword in d['lemmatized_text'].split(' ')]
len(sample)

мир


509

In [71]:
%%time
# Run the text through BERT, and collect all of the hidden states produced from all 12 layers. 
hidden_state_list = []
with torch.no_grad():
    for d in tqdm(sample):

        tokens_tensor = torch.tensor([d['doc']['input_ids']])
        segments_tensor = torch.tensor([d['doc']['attention_mask']])
        
        outputs = model(tokens_tensor, segments_tensor)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        hidden_states = outputs[2]
        hidden_state_list.append(hidden_states)

100%|████████████████████████████████████████| 509/509 [01:52<00:00,  4.51it/s]

CPU times: user 1min 45s, sys: 2.57 s, total: 1min 48s
Wall time: 1min 52s


In [76]:
random.choice(random.choice(hidden_state_list)).shape

torch.Size([1, 12, 768])

In [77]:
random.choice(sample)

{'doc': {'input_ids': [101, 15726, 58513, 869, 6913, 869, 84309, 64979, 128, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 'tokens': ['[CLS]',
  'играть',
  'мышь',
  'с',
  'мир',
  'с',
  'давний',
  'пора',
  ',',
  '[SEP]'],
 'text': 'играют мыши с миром с давних пор,',
 'lemmatized_text': 'играть мышь с мир с давний пора ,',
 'rec': {'Text': 'Смотри, сестра: вот мир на волоске,\nвот ручка Паркер в пляшущей руке\n(пусть не десница – но боготворят),\nвот росчерк, как полёт нетопыря,\nвот смятый лист под гербовой печатью,\nвот кабинет, в котором не заплачут\nнад миром, что повис на волоске.\n \nСмотри, сестра, у шахматной доски\nсвиваются клубками червяки.\nФигуры вот, что вскорости сожрут.\nСмерть некрасива даже на миру,\nно им опять про то сказать забыли,\nи потому лежат под слоем пыли\nте пешки, что уже снесли с доски.\n \nСмотри, сестра, вот мышь, а вот гора.\nМышь голодна, её кормить пора,\nно скоро у горы щедрот не с

In [85]:
# generate hidden states
print(keyword)
forjson = []
verbose = False
badsamples = []

for d_i, d in tqdm(enumerate(sample)):
    hidden_states = hidden_state_list[d_i]
    # create a new dimension in the tensor
    token_embeddings = torch.stack(hidden_states, dim=0)
    if verbose:
        print(token_embeddings.size())
    # Remove dimension 1, the "batches"
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    if verbose:
        print(token_embeddings.size())
    token_embeddings = token_embeddings.permute(1,0,2)
    if verbose:
        print(token_embeddings.size())
        
    keywordFound = False
    for t_i, token_str in enumerate(d['tokens']):
        if token_str == keyword:
            index = t_i
            keywordFound = True
            
    if not keywordFound:
        badsamples.append(d)
        continue
    
    # hiddenDict = dict()
    eachLayerDict = dict()
    for n in range(0,13):
        token_vecs_sum = []
        one_layer_only = []
        for token in token_embeddings:
            sum_vec = torch.sum(token[-n:], dim=0)
            token_vecs_sum.append(sum_vec)
            one_layer_only.append(token[n])
        # hiddenDict[n] = token_vecs_sum[index].tolist()
        eachLayerDict[n] = token_embeddings[index][n].tolist()

    forjson.append({
        'linetext' : d['text'],
        'lemmatized_text' : d['lemmatized_text'],
        'Author' : d['rec']['Author'],
        'fulltext' : d['rec']['Text'],
        'Before or after' : d['rec']['Before or after'],
        'Source' : d['rec']['Source'],
        'eachLayer' : eachLayerDict,
        # 'hiddenStates' : hiddenDict
    })
    
print(len(forjson))

мир


509it [00:05, 89.37it/s] 

509


In [92]:
random.choice(forjson)['eachLayer'][12][:10]

[-0.21235522627830505,
 -0.2960023283958435,
 0.36839088797569275,
 -0.0356932207942009,
 0.3543468415737152,
 0.4621066749095917,
 -0.46916061639785767,
 -1.1899771690368652,
 0.41594475507736206,
 1.336857795715332]

In [88]:
import json

with open('мир-1-30layers.json', 'w', encoding='utf-8') as f:
    json.dump(forjson, f, ensure_ascii=False, indent=4)

### Convert using PCA and TSNE

In [103]:
import numpy as np

#### PCA

In [104]:
# get last layer for all 
Xs = np.array([d['eachLayer'][12] for d in forjson])
Xs.shape

(509, 768)

In [111]:
# convert from 728 to 50
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(Xs.T)
pca.components_.shape

PCA(n_components=50)

In [118]:
Xs_PCA = pca.components_

#### TSNE

In [90]:
# convert 50 to 2

In [117]:
from sklearn.manifold import TSNE

In [ ]:
# X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(X)

In [ ]:
tsne = TSNE(n_components=2, random_state=0)
tsne.fit(Xs_PCA)
Xs_TSNE = tsne.embedding_
Xs_TSNE.shape

Below are a couple additional resources for exploring this topic.

**BERT Authors**

The BERT authors tested word-embedding strategies by feeding different vector combinations as input features to a BiLSTM used on a named entity recognition task and observing the resulting F1 scores.

(Image from [Jay Allamar](http://jalammar.github.io/illustrated-bert/)'s blog)


![alt text](http://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png)

While concatenation of the last four layers produced the best results on this specific task, many of the other methods come in a close second and in general it is advisable to test different versions for your specific application: results may vary.

This is partially demonstrated by noting that the different layers of BERT encode very different kinds of information, so the appropriate pooling strategy will change depending on the application because different layers encode different kinds of information. 


**Han Xiao's BERT-as-service**

Han Xiao created an open-source project named [bert-as-service](https://github.com/hanxiao/bert-as-service) on GitHub which is intended to create word embeddings for your text using BERT. Han experimented with different approaches to combining these embeddings, and shared some conclusions and rationale on the [FAQ page](https://github.com/hanxiao/bert-as-service#speech_balloon-faq) of the project. 

`bert-as-service`, by default, uses the outputs from the **second-to-last layer** of the model. 

I would summarize Han's perspective by the following:

1. The embeddings start out in the first layer as having no contextual information (i.e., the meaning of the initial 'bank' embedding isn't specific to river bank or financial bank).
2. As the embeddings move deeper into the network, they pick up more and more contextual information with each layer.
3. As you approach the final layer, however, you start picking up information that is specific to BERT's pre-training tasks (the "Masked Language Model" (MLM) and "Next Sentence Prediction" (NSP)). 
    * What we want is embeddings that encode the word meaning well... 
    * BERT is motivated to do this, but it is also motivated to encode anything else that would help it determine what a missing word is (MLM), or whether the second sentence came after the first (NSP). 
4. The second-to-last layer is what Han settled on as a reasonable sweet-spot.




## Cite
Chris McCormick and Nick Ryan. (2019, May 14). *BERT Word Embeddings Tutorial*. Retrieved from http://www.mccormickml.com
